In [1]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
pd.set_option('display.max_columns', None)

In [2]:
years = list(range(2016, 2023))

In [4]:
cols = ["home_team", "posteam", "posteam_type", "side_of_field", "yardline_100", "half_seconds_remaining", "game_seconds_remaining", "down",
       "ydstogo", "score_differential", "wp", "weather", "roof", "surface", "wind", "home_coach", "away_coach", "desc", "play_type", "pass_length",
       "air_yards", "run_location", "drive", "yrdln", "week", "season", "passer_player_name"]

In [5]:
nfl_df = nfl.import_pbp_data(years, columns = cols, cache = False, alt_path= None)

2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [6]:
nfl_df.shape

(341697, 41)

In [7]:
nfl_df.head()

,home_team,posteam,posteam_type,side_of_field,yardline_100,half_seconds_remaining,game_seconds_remaining,down,ydstogo,score_differential,wp,weather,roof,surface,wind,home_coach,away_coach,desc,play_type,pass_length,air_yards,run_location,drive,yrdln,week,passer_player_name,play_id,old_game_id,season,nflverse_game_id,possession_team,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
0,BAL,None,None,None,NaN,1800.0,3600.0,NaN,0.0,NaN,0.546262,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,GAME,None,None,NaN,None,NaN,BUF 35,1,None,1.0,2016091101,2016,2016_01_BUF_BAL,,None,None,NaN,None,NaN,,,,0.0,0.0
1,BAL,BAL,home,BUF,35.0,1800.0,3600.0,NaN,0.0,0.0,0.546262,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,4-Gay kicks 65 yards from BUF 35 to BAL 0. 14-...,kickoff,None,NaN,None,1.0,BUF 35,1,None,36.0,2016091101,2016,2016_01_BUF_BAL,BUF,None,None,NaN,None,NaN,40078;40151;37977;36060;40494;40053;40253;4011...,00-0030041;00-0030073;00-0029799;00-0030433;00...,00-0029892;00-0029895;00-0028497;00-0027539;00...,10.0,11.0
2,BAL,BAL,home,BAL,86.0,1797.0,3597.0,1.0,10.0,0.0,0.536378,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,(14:57) 29-J.Forsett right end pushed ob at BA...,run,None,NaN,right,1.0,BAL 14,1,None,58.0,2016091101,2016,2016_01_BUF_BAL,BAL,I_FORM,"2 RB, 1 TE, 2 WR",8.0,"4 DL, 3 LB, 4 DB",NaN,38540;41302;40078;35553;38582;43295;40053;4336...,00-0029892;00-0027714;00-0032965;00-0029893;00...,00-0029542;00-0031171;00-0029566;00-0030073;00...,11.0,11.0
3,BAL,BAL,home,BAL,80.0,1772.0,3572.0,2.0,4.0,0.0,0.541975,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,"(14:32) (No Huddle, Shotgun) 5-J.Flacco pass s...",pass,short,-4.0,None,1.0,BAL 20,1,J.Flacco,85.0,2016091101,2016,2016_01_BUF_BAL,BAL,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",3.0,41302;40078;38540;35553;43295;38582;40053;4336...,00-0029892;00-0027714;00-0032965;00-0029893;00...,00-0031171;00-0029542;00-0029566;00-0030073;00...,11.0,11.0
4,BAL,BAL,home,BAL,75.0,1741.0,3541.0,1.0,10.0,0.0,0.546262,"Partly Cloudy Temp: 84° F, Humidity: 44%, Wind...",outdoors,grass,8.0,John Harbaugh,Rex Ryan,"(14:01) (No Huddle, Shotgun) 29-J.Forsett left...",run,None,NaN,left,1.0,BAL 25,1,None,109.0,2016091101,2016,2016_01_BUF_BAL,BAL,SHOTGUN,"2 RB, 1 TE, 2 WR",7.0,"4 DL, 3 LB, 4 DB",NaN,40078;38540;41302;35553;43295;38582;40053;4336...,00-0029892;00-0027714;00-0032965;00-0029893;00...,00-0029542;00-0031171;00-0029566;00-0030073;00...,11.0,11.0


In [8]:
nfl_clean = nfl_df[(nfl_df['play_type'] == 'pass') | (nfl_df['play_type'] == 'run')].reset_index(drop=True)

In [9]:
nfl_clean.shape

(239535, 41)

In [10]:
# make yardline yards to distance to endzone
yards_to_end = []
for i in range(len(nfl_clean)):
    if nfl_clean['posteam'][i] == nfl_clean["yrdln"][i].split(" ")[0]:
        yards_to_end.append(100 - int(nfl_clean['yrdln'][i].split(" ")[1]))
    else:
        yards_to_end.append(nfl_clean['yrdln'][i].split(" ")[1])
        

In [11]:
nfl_clean['yds_to_endzone'] = yards_to_end

In [12]:
# check for rain
rain_type = ['Rain', 'Raining', 'Rainy', 'Showers']
weather_type=[]
for w in nfl_clean['weather']:
    if w:
        if w.split(" ")[0] in rain_type:
            weather_type.append("Rain")
        elif w.split(" ")[0] == "Snow":
            weather_type.append("Snow")
        else:
            weather_type.append("None")
    else:
        weather_type.append("None")

In [13]:
nfl_clean['Precipitation'] = weather_type

In [14]:
# head coach on offense
offense_hc = []
for i in range(len(nfl_clean)):
    if nfl_clean['posteam_type'][i] == 'home':
        offense_hc.append(nfl_clean['home_coach'][i])
    else:
        offense_hc.append(nfl_clean['away_coach'][i])


In [15]:
nfl_clean['Offense_HC'] = offense_hc

In [16]:
# replace missing roof with outdoors (None values)
nfl_clean['roof'] = nfl_clean['roof'].fillna("outdoors")

In [17]:
# use indoor or outdoor for roof values
indoor_type = ['closed', 'dome']
outdoor_type = ['open', 'outdoors']
indoors = []
for i in range(len(nfl_clean)):
    if nfl_clean['roof'][i] in indoor_type:
        indoors.append(True)
    else:
        indoors.append(False)
nfl_clean['Indoors'] = indoors

In [18]:
# replace NaN in wind with 0
nfl_clean['wind'] = nfl_clean['wind'].fillna(0)

In [19]:
# replace NaN in down with 1 (two point conversions so effectively not kicking situation)
nfl_clean['down'] = nfl_clean['down'].fillna(1.0)

In [20]:
nfl_clean = nfl_clean.dropna(subset=['nflverse_game_id', 'drive','offense_personnel', 'defense_personnel'])

In [43]:
nfl_clean1 = nfl_clean.drop(["side_of_field", "yardline_100", "weather", "home_coach", "away_coach", "play_id", 
                            "old_game_id", "possession_team", "players_on_play", "offense_players",
                            "defense_players", "n_offense", "n_defense", "number_of_pass_rushers","yrdln", "roof"], axis = 1)

In [44]:
nfl_clean1.isna().sum()

home_team                      0
posteam                        0
posteam_type                   0
half_seconds_remaining         0
game_seconds_remaining         0
down                           0
ydstogo                        0
score_differential             0
wp                             0
surface                        0
wind                           0
desc                           0
play_type                      0
pass_length               107393
air_yards                 107364
run_location              141533
drive                          0
week                           0
passer_player_name         97841
season                         0
nflverse_game_id               0
offense_formation            304
offense_personnel              0
defenders_in_box             235
defense_personnel              0
yds_to_endzone                 0
Precipitation                  0
Offense_HC                     0
Indoors                        0
dtype: int64

In [45]:
import re

In [46]:
# make num rbs and num tes columns
rbs = []
tes = []
off_personnel = []
for p in nfl_clean1['offense_personnel']:
    if p:
        rbs.append(int(re.search('. RB', p)[0].split(" ")[0]))
        tes.append(int(re.search('. TE', p)[0].split(" ")[0]))
    else:
        tes.append(None)
        rbs.append(None)

nfl_clean1['num_rbs'] = rbs
nfl_clean1['num_tes'] = tes


In [47]:
# make num dls and lbs columns
dls = []
lbs = []
for d in nfl_clean1['defense_personnel']:
    if d:
        dls.append(int(re.search('. DL', d)[0].split(" ")[0]))
        lbs.append(int(re.search('. LB', d)[0].split(" ")[0]))
    else:
        dls.append(None)
        lbs.append(None)
nfl_clean1['num_dl'] = dls
nfl_clean1['num_lbs'] = lbs

In [48]:
nfl_clean1 = nfl_clean1.dropna(subset=['offense_formation', 'defenders_in_box', 'num_rbs'])

In [49]:
nfl_clean1.isna().sum()

home_team                      0
posteam                        0
posteam_type                   0
half_seconds_remaining         0
game_seconds_remaining         0
down                           0
ydstogo                        0
score_differential             0
wp                             0
surface                        0
wind                           0
desc                           0
play_type                      0
pass_length               107203
air_yards                 107174
run_location              141379
drive                          0
week                           0
passer_player_name         97659
season                         0
nflverse_game_id               0
offense_formation              0
offense_personnel              0
defenders_in_box               0
defense_personnel              0
yds_to_endzone                 0
Precipitation                  0
Offense_HC                     0
Indoors                        0
num_rbs                        0
num_tes   

In [50]:
# add previous season qbr

In [51]:
qbr = nfl.import_qbr([2015, 2016, 2017, 2018, 2019, 2020, 2021], level = "nfl")

In [52]:
qbr = qbr[qbr['season_type'] == 'Regular'].reset_index()
qbr = qbr[['season', 'name_short', 'qbr_total']]
# change qbr name to match format in other table
for i in range(len(qbr)):
    qbr['name_short'][i] = qbr['name_short'][i].split(" ")[0] + qbr['name_short'][i].split(" ")[1]

/var/folders/ky/t3qz7fy91yx5vrk7bkh4xyr00000gn/T/ipykernel_74547/2001523682.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qbr['name_short'][i] = qbr['name_short'][i].split(" ")[0] + qbr['name_short'][i].split(" ")[1]
/var/folders/ky/t3qz7fy91yx5vrk7bkh4xyr00000gn/T/ipykernel_74547/2001523682.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qbr['name_short'][i] = qbr['name_short'][i].split(" ")[0] + qbr['name_short'][i].split(" ")[1]
/var/folders/ky/t3qz7fy91yx5vrk7bkh4xyr00000gn/T/ipykernel_74547/2001523682.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [53]:
qb_weekly = nfl_clean1.groupby(by=["season","posteam", "week"]).first().reset_index()

In [54]:
qb_week = qb_weekly[['posteam', 'week','passer_player_name', 'season']]
qb_week['prev_season'] = qb_week['season'] - 1

/var/folders/ky/t3qz7fy91yx5vrk7bkh4xyr00000gn/T/ipykernel_74547/652913706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qb_week['prev_season'] = qb_week['season'] - 1


In [55]:
merged = qb_week.merge(qbr, how='left', left_on = ['prev_season', 'passer_player_name'], 
                       right_on = ['season', 'name_short'])
merged.shape

(3804, 8)

In [56]:
qbr_median = qbr['qbr_total'].median()
merged['qbr_total'] = merged['qbr_total'].fillna(qbr_median)

In [57]:
merged = merged.drop(['season_y', 'name_short'], axis = 1)

In [58]:
nfl_clean2 = nfl_clean1.merge(merged, left_on =['season', 'posteam', 'week'], 
                              right_on =['season_x', 'posteam', 'week'])

In [59]:
nfl_clean2.shape

(238098, 37)

In [60]:
nfl_clean2.isna().sum()

home_team                      0
posteam                        0
posteam_type                   0
half_seconds_remaining         0
game_seconds_remaining         0
down                           0
ydstogo                        0
score_differential             0
wp                             0
surface                        0
wind                           0
desc                           0
play_type                      0
pass_length               107203
air_yards                 107174
run_location              141379
drive                          0
week                           0
passer_player_name_x       97659
season                         0
nflverse_game_id               0
offense_formation              0
offense_personnel              0
defenders_in_box               0
defense_personnel              0
yds_to_endzone                 0
Precipitation                  0
Offense_HC                     0
Indoors                        0
num_rbs                        0
num_tes   

In [61]:
nfl_final = nfl_clean2.drop(['season', 'home_team', 'pass_length', 'air_yards', 'run_location', 'desc', 'nflverse_game_id',
                             'offense_personnel', 'defense_personnel','passer_player_name_x', 'week', 'season', 'season_x', 'passer_player_name_y',
                             'prev_season'], axis = 1)

In [62]:
nfl_final.shape

(238098, 23)

In [63]:
nfl_final.isna().sum()

posteam                   0
posteam_type              0
half_seconds_remaining    0
game_seconds_remaining    0
down                      0
ydstogo                   0
score_differential        0
wp                        0
surface                   0
wind                      0
play_type                 0
drive                     0
offense_formation         0
defenders_in_box          0
yds_to_endzone            0
Precipitation             0
Offense_HC                0
Indoors                   0
num_rbs                   0
num_tes                   0
num_dl                    0
num_lbs                   0
qbr_total                 0
dtype: int64

In [77]:
nfl_final['play_type'] = nfl_final['play_type'].replace({'run':0, 'pass':1})

In [113]:
nfl_final['posteam'] = pd.Categorical(nfl_final.posteam)
nfl_final['posteam_type'] = pd.Categorical(nfl_final.posteam_type)
nfl_final['surface'] = pd.Categorical(nfl_final.surface)
nfl_final['offense_formation'] = pd.Categorical(nfl_final.offense_formation)
nfl_final['Precipitation'] = pd.Categorical(nfl_final.Precipitation)
nfl_final['yds_to_endzone'] = pd.to_numeric(nfl_final.yds_to_endzone)
nfl_final['Offense_HC'] = pd.Categorical(nfl_final.Offense_HC)

In [114]:
nfl_final.dtypes

posteam                   category
posteam_type              category
half_seconds_remaining     float32
game_seconds_remaining     float32
down                       float32
ydstogo                    float32
score_differential         float32
wp                         float32
surface                   category
wind                       float32
play_type                    int64
drive                      float32
offense_formation         category
defenders_in_box           float32
yds_to_endzone               int64
Precipitation             category
Offense_HC                category
Indoors                       bool
num_rbs                    float64
num_tes                    float64
num_dl                     float64
num_lbs                    float64
qbr_total                  float64
dtype: object

In [ ]:
# input

In [115]:
nfl_input = nfl_final.drop(['posteam', 'play_type'], axis = 1)

In [116]:
# output

In [117]:
nfl_output = nfl_final['play_type']

In [118]:
nfl_output.head(10)

0    0
1    1
2    0
3    1
4    0
5    1
6    1
7    1
8    1
9    0
Name: play_type, dtype: int64

In [119]:
# Test-Train Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(nfl_input, nfl_output, test_size = 0.25, random_state = 22)

In [120]:
import xgboost as xgb

In [121]:
train = xgb.DMatrix(X_train, label = y_train, enable_categorical = True)
test = xgb.DMatrix(X_test, label = y_test, enable_categorical = True)

/Users/matthewsasaki/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  or is_sparse(dtype)
/Users/matthewsasaki/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)
/Users/matthewsasaki/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:374: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  or (is_nullable_dtype(dtype) and not is_categorical_dtype(dtype))
/Users/matthewsasaki/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:375: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use 

In [135]:
param = {
    'max_depth':7,
    'eta':0.2,
    'objective':'binary:hinge',
}
epochs = 10

In [136]:
model = xgb.train(param, train, epochs)

In [137]:
predictions = model.predict(test)

In [138]:
print(predictions)

[0. 0. 1. ... 1. 0. 0.]


In [139]:
from sklearn.metrics import accuracy_score

In [140]:
accuracy_score(y_test, predictions)

0.7209239815203696